# Description

A sophisticated approach to training Convolutional Neural Networks (CNNs) in a distributed environment is federated learning with PyTorch. Clients receive data and train their version of the model locally. Afterwards, the models are combined using federated averaging, creating a stronger global model. A unified test set is used to evaluate performance, and metrics such as loss, accuracy, and confusion matrix are employed for analysis.

Referência: [PYTORCH DOCUMENTATION](https://pytorch.org/docs/stable/index.html)
Referência: [PyTorch Tutorials](https://pytorch.org/tutorials/)
Referência: [PySyft’s documentation](https://openmined.github.io/PySyft/index.html)
Referência: [Hello PyTorch: NVIDIA](https://nvflare.readthedocs.io/en/main/examples/hello_pt.html)

Autor: Ernesto Gurgel Valente Neto



# Fed-learning

Detailed implementation of a federated learning system using PyTorch, aimed at training a Convolutional Neural Network (CNN).
**Structure and Functionality:**

*   **Federated Data Distribution:** The training dataset is divided among multiple clients to simulate a federated environment, where each client possesses a portion of the data.

*   **Data Preparation:** Utilizes transformations and normalizations to prepare the training and testing datasets for efficient network training.

*   **Local Training and Evaluation:** Each client trains its model locally using its own data. After training, the models are evaluated to check the performance before aggregation.

*   **Model Aggregation (aggregate_models):** The locally trained models are aggregated into a single global model using the federated averaging technique, which combines the weights of the models to form an updated and potentially more robust model.

*   **Evaluation of Global Model:** The resulting global model is tested using a unified test dataset to evaluate its overall performance and ensure it generalizes well on unseen data.

*   **Metrics Visualization:** Features for visualizing training progress, including loss and accuracy over time, as well as the confusion matrix for detailed analysis of model performance.

*   **CNN Architecture:** CNN architecture for image processing. The networks utilize convolutional layers to extract features from images, followed by dense layers that classify the images.


### FUNCTIONS TO FED-ENVIRONMENT.

The main objective functions generate subsets of data that are representative of NON-IID federated learning environments, simulating a situation where different users have data from different classes and in varying quantities.

*   NON-IID FUNCTIONS
*   FED-LEARNING FUNCTIONS


#### NON-IID FUNCTIONS

Each function starts by calculating the number of "shards" or data segments available based on the size of the original dataset and the number of samples per class.
Users receive data from a specific number of randomly chosen classes, ensuring that each user has a unique and diversified subset of data in terms of classes.
Sample selection for each class is done randomly without replacement, ensuring no duplicates in the samples distributed to each user.

##### [NO FILTER] IDD&NON-IDD FUNCTIONS
*   Creates a federated environment with balanced and unbalanced data distributions.


In [ ]:
def MNISTNonIID(train_dataset, num_users, n_class=2, num_samples=3000, rate_unbalance=0):
    # Parâmetros:
    # - num_users: Número de usuários/clientes.
    # - n_class: Número de classes por usuário.
    # - num_samples: Número base de amostras por classe para cada usuário.
    # - rate_unbalance: Fator de desequilíbrio na distribuição das amostras.
    num_shards_train = int(60000 / num_samples)
    num_classes = 10  # Total de classes
    assert(n_class * num_users <= num_shards_train), "There are not enough shards to allocate"
    assert(n_class <= num_classes), "Number of classes per user greater than the total available classes"
    labels = np.array(train_dataset.targets)
    class_indices = [np.where(labels == i)[0] for i in range(num_classes)]
    users_dict_train = {i: np.array([]) for i in range(num_users)}
    for user in range(num_users):
        chosen_classes = np.random.choice(range(num_classes), n_class, replace=False)
        for cls in chosen_classes:
            idxs = np.random.choice(class_indices[cls], num_samples, replace=False)
            users_dict_train[user] = np.concatenate((users_dict_train[user], idxs), axis=0)
    client_datasets = [Subset(train_dataset, indices.astype(int)) for indices in users_dict_train.values()]
    return client_datasets

def FashionMNISTNonIID(train_dataset, num_users, n_class=2, num_samples=3000, rate_unbalance=0):
    # Parâmetros:
    # - num_users: Número de usuários/clientes.
    # - n_class: Número de classes por usuário.
    # - num_samples: Número base de amostras por classe para cada usuário.
    # - rate_unbalance: Fator de desequilíbrio na distribuição das amostras.
    num_shards_train = int(60000 / num_samples)
    num_classes = 10  # Total de classes
    assert(n_class * num_users <= num_shards_train), "There are not enough shards to allocate"
    assert(n_class <= num_classes), "Number of classes per user greater than the total available classes"
    labels = np.array(train_dataset.targets)
    class_indices = [np.where(labels == i)[0] for i in range(num_classes)]
    users_dict_train = {i: np.array([]) for i in range(num_users)}
    for user in range(num_users):
        chosen_classes = np.random.choice(range(num_classes), n_class, replace=False)
        for cls in chosen_classes:
            idxs = np.random.choice(class_indices[cls], num_samples, replace=False)
            users_dict_train[user] = np.concatenate((users_dict_train[user], idxs), axis=0)
    client_datasets = [Subset(train_dataset, indices.astype(int)) for indices in users_dict_train.values()]
    return client_datasets

def Cifar10NonIID(train_dataset, num_users, n_class=2, num_samples=2500, rate_unbalance=0):
    # Parâmetros:
    # - num_users: Número de usuários/clientes.
    # - n_class: Número de classes por usuário.
    # - num_samples: Número base de amostras por classe para cada usuário.
    # - rate_unbalance: Fator de desequilíbrio na distribuição das amostras.
    num_shards_train = int(50000 / num_samples)
    num_classes = len(np.unique(train_dataset.targets))
    assert(n_class * num_users <= num_shards_train), "There are not enough shards to allocate"
    assert(n_class <= num_classes), "Number of classes per user greater than the total available classes"
    # Preparação dos índices de dados
    labels = np.array(train_dataset.targets)
    class_indices = [np.where(labels == i)[0] for i in range(num_classes)]
    # Dicionário para armazenar os índices dos dados para cada usuário
    users_dict = {i: np.array([]) for i in range(num_users)}
    for user in range(num_users):
        user_labels = np.array([])
        # Seleciona classes de forma aleatória e sem reposição para garantir diversidade
        chosen_classes = np.random.choice(range(num_classes), n_class, replace=False)
        for cls in chosen_classes:
            idxs = np.random.choice(class_indices[cls], num_samples, replace=False)
            users_dict[user] = np.concatenate((users_dict[user], idxs), axis=0)
    # Criar datasets para cada cliente baseado nos índices selecionados
    client_datasets = [Subset(train_dataset, indices.astype(int)) for indices in users_dict.values()]
    return client_datasets

def Cifar100NonIID(train_dataset, num_users, n_class=20, num_samples=500, rate_unbalance=0):
    # Parâmetros:
    # - num_users: Número de usuários/clientes.
    # - n_class: Número de classes por usuário.
    # - num_samples: Número base de amostras por classe para cada usuário.
    # - rate_unbalance: Fator de desequilíbrio na distribuição das amostras.
    num_shards_train = int(50000 / num_samples)  # Baseado no tamanho do CIFAR-100
    num_classes = 100  # Total de classes no CIFAR-100
    assert(n_class * num_users <= num_shards_train), "There are not enough shards to allocate"
    assert(n_class <= num_classes), "Number of classes per user greater than the total available classes"
    # Preparação dos índices de dados
    labels = np.array(train_dataset.targets)
    class_indices = [np.where(labels == i)[0] for i in range(num_classes)]
    # Dicionário para armazenar os índices dos dados para cada usuário
    users_dict = {i: np.array([]) for i in range(num_users)}
    for user in range(num_users):
        # Escolha aleatória de classes para garantir diversidade
        chosen_classes = np.random.choice(range(num_classes), n_class, replace=False)
        for cls in chosen_classes:
            idxs = np.random.choice(class_indices[cls], num_samples, replace=False)
            users_dict[user] = np.concatenate((users_dict[user], idxs), axis=0)
    # Criar datasets para cada cliente baseado nos índices selecionados
    client_datasets = [Subset(train_dataset, indices.astype(int)) for indices in users_dict.values()]
    return client_datasets

# Parâmetros:
# - num_users: Número de usuários/clientes.
# - n_class: Número de classes por usuário.
# - num_samples: Número base de amostras por classe para cada usuário.
# - rate_unbalance: Fator de desequilíbrio na distribuição das amostras.
# - Alto fator de desbalanceamento de quanitade de classes e dados (DADOS NON-IDD).

def MNISTNonIIDunbalance(train_dataset, num_users, n_class=2, num_samples=3000, rate_unbalance=1.2):
    num_classes = 10  # Total de classes para MNIST
    labels = np.array(train_dataset.targets)
    class_indices = [np.where(labels == i)[0] for i in range(num_classes)]
    users_dict_train = {i: np.array([]) for i in range(num_users)}
    # Calcula o número de amostras por usuário considerando o desequilíbrio
    for user in range(num_users):
        chosen_classes = np.random.choice(range(num_classes), n_class, replace=False)
        # Aplica o rate_unbalance de forma cumulativa
        user_samples = int(num_samples * (rate_unbalance ** user))
        for cls in chosen_classes:
            available_samples = min(user_samples, len(class_indices[cls]))
            idxs = np.random.choice(class_indices[cls], available_samples, replace=False)
            users_dict_train[user] = np.concatenate((users_dict_train[user], idxs), axis=0)
    client_datasets = [Subset(train_dataset, indices.astype(int)) for indices in users_dict_train.values()]
    return client_datasets

def FashionMNISTNonIIDunbalance(train_dataset, num_users, n_class=2, num_samples=3000, rate_unbalance=1.2):
    num_classes = 10  # Total de classes para FashionMNIST
    labels = np.array(train_dataset.targets)
    class_indices = [np.where(labels == i)[0] for i in range(num_classes)]
    users_dict_train = {i: np.array([]) for i in range(num_users)}
    for user in range(num_users):
        chosen_classes = np.random.choice(range(num_classes), n_class, replace=False)
        user_samples = int(num_samples * (rate_unbalance ** user))
        for cls in chosen_classes:
            available_samples = min(user_samples, len(class_indices[cls]))
            idxs = np.random.choice(class_indices[cls], available_samples, replace=False)
            users_dict_train[user] = np.concatenate((users_dict_train[user], idxs), axis=0)
    client_datasets = [Subset(train_dataset, indices.astype(int)) for indices in users_dict_train.values()]
    return client_datasets

def Cifar10NonIIDunbalance(train_dataset, num_users, n_class=2, num_samples=2500, rate_unbalance=1.2):
    num_classes = len(np.unique(train_dataset.targets))  # Total de classes para CIFAR10
    labels = np.array(train_dataset.targets)
    class_indices = [np.where(labels == i)[0] for i in range(num_classes)]
    users_dict = {i: np.array([]) for i in range(num_users)}
    for user in range(num_users):
        chosen_classes = np.random.choice(range(num_classes), n_class, replace=False)
        user_samples = int(num_samples * (rate_unbalance ** user))
        for cls in chosen_classes:
            available_samples = min(user_samples, len(class_indices[cls]))
            idxs = np.random.choice(class_indices[cls], available_samples, replace=False)
            users_dict[user] = np.concatenate((users_dict[user], idxs), axis=0)
    client_datasets = [Subset(train_dataset, indices.astype(int)) for indices in users_dict.values()]
    return client_datasets

def Cifar100NonIIDunbalance(train_dataset, num_users, n_class=20, num_samples=500, rate_unbalance=1.2):
    num_classes = 100  # Total de classes para CIFAR100
    labels = np.array(train_dataset.targets)
    class_indices = [np.where(labels == i)[0] for i in range(num_classes)]
    users_dict = {i: np.array([]) for i in range(num_users)}
    for user in range(num_users):
        chosen_classes = np.random.choice(range(num_classes), n_class, replace=False)
        user_samples = int(num_samples * (rate_unbalance ** user))
        for cls in chosen_classes:
            available_samples = min(user_samples, len(class_indices[cls]))
            idxs = np.random.choice(class_indices[cls], available_samples, replace=False)
            users_dict[user] = np.concatenate((users_dict[user], idxs), axis=0)
    client_datasets = [Subset(train_dataset, indices.astype(int)) for indices in users_dict.values()]
    return client_datasets

##### [ENTROPY FILTER] IDD&NON-IDD FUNCTIONS
*   Creates a federated environment with balanced and unbalanced data distributions where entropy is applied before training.


In [ ]:
# Funções de entropia, calcula a entropia de cada classe individualmente
# Seleciona as imagens de acordo com a distribuição de entropia
# Recebe o numero de classes e os conjuntos de dados de train_X e train_y.
import numpy as np
import time

def entropia(pk, base=2):
    pk = pk / np.sum(pk)
    pk = pk[pk > 0]
    result = -np.sum(pk * np.log(pk) / np.log(base))
    return result

def filtrar_entropia_classe(train_X, train_y, classes_relevantes):
    start_time = time.time()  # Inicia a contagem de tempo
    train_Xextend = []
    train_yextend = []
    indices_filtrados_extend = []
    for label in classes_relevantes: #Se existe amostras para a classe atual
        indices_originais = np.where(train_y == label)[0]
        if len(indices_originais) == 0:
            continue  # Se não há amostras para a classe atual
        indicesDasImagensDaClasse = train_X[indices_originais]
        tuplas = [(indice, entropia(img)) for indice, img in zip(indices_originais, indicesDasImagensDaClasse)]
        local_ordenado = sorted(tuplas, key=lambda x: x[1])
        n = len(local_ordenado)
        if n % 2 == 1:
            median = local_ordenado[n // 2][1]
        else:
            median = (local_ordenado[n // 2 - 1][1] + local_ordenado[n // 2][1]) / 2.0
        indices_filtrados_da_classe = [item[0] for item in local_ordenado if item[1] <= median]
        train_Xextend.extend(train_X[indice] for indice in indices_filtrados_da_classe)
        train_yextend.extend(train_y[indice] for indice in indices_filtrados_da_classe)
        indices_filtrados_extend.extend(indices_filtrados_da_classe)
    end_time = time.time()  # Finaliza a contagem de tempo
    print(f"Total entropy execution time: {end_time - start_time} seconds.")
    return np.array(train_Xextend), np.array(train_yextend), np.array(indices_filtrados_extend)

    # Parâmetros:
    # - num_users: Número de usuários/clientes.
    # - n_class: Número de classes por usuário.
    # - num_samples: Número base de amostras por classe para cada usuário.
    # - rate_unbalance: Fator de desequilíbrio na distribuição das amostras.
def MNISTNonIID_filtrado(train_dataset, num_users, n_class=2, num_samples=3000, rate_unbalance=0):
    num_shards_train = int(60000 / num_samples)
    num_classes = 10  # Total de classes para MNIST
    assert(n_class * num_users <= num_shards_train), "There are not enough shards to allocate"
    assert(n_class <= num_classes), "Number of classes per user greater than the total available classes"
    labels = np.array(train_dataset.targets)
    class_indices = [np.where(labels == i)[0] for i in range(num_classes)]
    users_dict_train = {i: np.array([]) for i in range(num_users)}
    chosen_classes_per_user = {}  # Novo dicionário para armazenar as classes escolhidas para cada usuário
    for user in range(num_users):
        chosen_classes = np.random.choice(range(num_classes), n_class, replace=False)
        chosen_classes_per_user[user] = chosen_classes  # Armazena as classes escolhidas para o usuário
        for cls in chosen_classes:
            idxs = np.random.choice(class_indices[cls], num_samples, replace=False)
            users_dict_train[user] = np.concatenate((users_dict_train[user], idxs), axis=0)
    client_datasets_filtrados = []
    for user, indices in users_dict_train.items():
        train_X = np.array([train_dataset.data[int(i)].numpy() for i in indices])
        train_y = np.array([train_dataset.targets[int(i)] for i in indices])
        # Aplicar filtragem baseada em entropia
        train_X_filtrado, train_y_filtrado, indices_filtrados = filtrar_entropia_classe(train_X, train_y, chosen_classes_per_user[user])  # Passa as classes escolhidas
        # Corrige para usar os índices filtrados corretamente
        indices_filtrados_absolutos = [int(indices[int(i)]) for i in indices_filtrados]
        client_datasets_filtrados.append(Subset(train_dataset, indices_filtrados_absolutos))
    return client_datasets_filtrados

def FashionMNISTNonIID_filtrado(train_dataset, num_users, n_class=2, num_samples=3000, rate_unbalance=0):
    num_shards_train = int(60000 / num_samples)
    num_classes = 10  # Total de classes
    assert(n_class * num_users <= num_shards_train), "There are not enough shards to allocate"
    assert(n_class <= num_classes), "Number of classes per user greater than the total available classes"
    labels = np.array(train_dataset.targets)
    class_indices = [np.where(labels == i)[0] for i in range(num_classes)]
    users_dict_train = {i: np.array([]) for i in range(num_users)}
    chosen_classes_per_user = {}
    for user in range(num_users):
        chosen_classes = np.random.choice(range(num_classes), n_class, replace=False)
        chosen_classes_per_user[user] = chosen_classes
        for cls in chosen_classes:
            idxs = np.random.choice(class_indices[cls], num_samples, replace=False)
            users_dict_train[user] = np.concatenate((users_dict_train[user], idxs), axis=0)
    client_datasets_filtrados = []
    for user, indices in users_dict_train.items():
        train_X = np.array([train_dataset.data[int(i)].numpy() for i in indices])
        train_y = np.array([train_dataset.targets[int(i)] for i in indices])
        train_X_filtrado, train_y_filtrado, indices_filtrados = filtrar_entropia_classe(train_X, train_y, chosen_classes_per_user[user])
        # Corrige para usar os índices filtrados corretamente
        indices_filtrados_absolutos = [int(indices[int(i)]) for i in indices_filtrados]
        client_datasets_filtrados.append(Subset(train_dataset, indices_filtrados_absolutos))
    return client_datasets_filtrados

def Cifar10NonIID_filtrado(train_dataset, num_users, n_class=2, num_samples=2500, rate_unbalance=0):
    num_shards_train = int(50000 / num_samples)  # Baseado no tamanho do CIFAR-10
    num_classes = len(np.unique(train_dataset.targets))  # Total de classes no CIFAR-10
    assert(n_class * num_users <= num_shards_train), "There are not enough shards to allocate"
    assert(n_class <= num_classes), "Number of classes per user greater than the total available classes"
    labels = np.array(train_dataset.targets)
    class_indices = [np.where(labels == i)[0] for i in range(num_classes)]
    users_dict = {i: np.array([]) for i in range(num_users)}  # Corrigido para users_dict
    chosen_classes_per_user = {}  # Dicionário para armazenar as classes escolhidas para cada usuário
    for user in range(num_users):
        chosen_classes = np.random.choice(range(num_classes), n_class, replace=False)  # Escolhe n_class classes aleatoriamente
        chosen_classes_per_user[user] = chosen_classes  # Armazena as classes escolhidas para este usuário
        for cls in chosen_classes:
            idxs = np.random.choice(class_indices[cls], num_samples, replace=False)  # Escolhe num_samples índices para cada classe
            users_dict[user] = np.concatenate((users_dict[user], idxs), axis=0)  # Concatena os índices no dicionário do usuário
    client_datasets_filtrados = []
    for user, indices in users_dict.items():
        #train_X = np.array([train_dataset.data[int(i)].numpy() for i in indices])
        train_X = np.array([train_dataset.data[int(i)] for i in indices])
        train_y = np.array([train_dataset.targets[int(i)] for i in indices])
        # Aplicar filtragem baseada em entropia
        train_X_filtrado, train_y_filtrado, indices_filtrados = filtrar_entropia_classe(train_X, train_y, chosen_classes_per_user[user])  # Passa as classes escolhidas para a função
        # Corrige para usar os índices filtrados corretamente
        indices_filtrados_absolutos = [int(indices[int(i)]) for i in indices_filtrados]
        client_datasets_filtrados.append(Subset(train_dataset, indices_filtrados_absolutos))
    return client_datasets_filtrados

def Cifar100NonIID_filtrado(train_dataset, num_users, n_class=20, num_samples=250, rate_unbalance=0):
    num_shards_train = int(50000 / num_samples)
    num_classes = 100  # Total de classes no CIFAR-100
    assert(n_class * num_users <= num_shards_train), "There are not enough shards to allocate"
    assert(n_class <= num_classes), "Number of classes per user greater than the total available classes"
    labels = np.array(train_dataset.targets)
    class_indices = [np.where(labels == i)[0] for i in range(num_classes)]
    users_dict = {i: np.array([]) for i in range(num_users)}
    chosen_classes_per_user = {}
    for user in range(num_users):
        chosen_classes = np.random.choice(range(num_classes), n_class, replace=False)
        chosen_classes_per_user[user] = chosen_classes
        for cls in chosen_classes:
            idxs = np.random.choice(class_indices[cls], num_samples, replace=False)
            users_dict[user] = np.concatenate((users_dict[user], idxs), axis=0)
    client_datasets_filtrados = []
    for user, indices in users_dict.items():
        #train_X = np.array([train_dataset.data[int(i)].numpy() for i in indices])
        train_X = np.array([train_dataset.data[int(i)] for i in indices])
        train_y = np.array([train_dataset.targets[int(i)] for i in indices])
        train_X_filtrado, train_y_filtrado, indices_filtrados = filtrar_entropia_classe(train_X, train_y, chosen_classes_per_user[user])
        # Corrige para usar os índices filtrados corretamente
        indices_filtrados_absolutos = [int(indices[int(i)]) for i in indices_filtrados]
        client_datasets_filtrados.append(Subset(train_dataset, indices_filtrados_absolutos))
    return client_datasets_filtrados

# Parâmetros:
# - num_users: Número de usuários/clientes.
# - n_class: Número de classes por usuário.
# - num_samples: Número base de amostras por classe para cada usuário.
# - rate_unbalance: Fator de desequilíbrio na distribuição das amostras.
# - Alto fator de desbalanceamento de quanitade de classes e dados (DADOS NON-IDD).
def MNISTNonIIDunbalance_filtrado(train_dataset, num_users, n_class=2, num_samples=3000, rate_unbalance=1.2):
    num_classes = 10  # Total de classes para MNIST
    labels = np.array(train_dataset.targets)
    class_indices = [np.where(labels == i)[0] for i in range(num_classes)]
    users_dict_train = {i: np.array([]) for i in range(num_users)}
    chosen_classes_per_user = {}
    for user in range(num_users):
        chosen_classes = np.random.choice(range(num_classes), n_class, replace=False)
        chosen_classes_per_user[user] = chosen_classes
        user_samples = int(num_samples * (rate_unbalance ** user))
        for cls in chosen_classes:
            available_samples = min(user_samples, len(class_indices[cls]))
            idxs = np.random.choice(class_indices[cls], available_samples, replace=False)
            users_dict_train[user] = np.concatenate((users_dict_train[user], idxs), axis=0)
    client_datasets_filtrados = []
    for user, indices in users_dict_train.items():
        train_X = np.array([train_dataset.data[int(i)].numpy() for i in indices])
        train_y = np.array([train_dataset.targets[int(i)] for i in indices])
        train_X_filtrado, train_y_filtrado, indices_filtrados = filtrar_entropia_classe(train_X, train_y, chosen_classes_per_user[user])
        # Corrige para usar os índices filtrados corretamente
        indices_filtrados_absolutos = [int(indices[int(i)]) for i in indices_filtrados]
        client_datasets_filtrados.append(Subset(train_dataset, indices_filtrados_absolutos))
    return client_datasets_filtrados

def FashionMNISTNonIIDunbalance_filtrado(train_dataset, num_users, n_class=2, num_samples=3000, rate_unbalance=1.2):
    num_classes = 10  # Total de classes para FashionMNIST
    labels = np.array(train_dataset.targets)
    class_indices = [np.where(labels == i)[0] for i in range(num_classes)]
    users_dict_train = {i: np.array([]) for i in range(num_users)}
    chosen_classes_per_user = {}
    for user in range(num_users):
        chosen_classes = np.random.choice(range(num_classes), n_class, replace=False)
        chosen_classes_per_user[user] = chosen_classes
        user_samples = int(num_samples * (rate_unbalance ** user))  # Ajuste para desbalanceamento
        for cls in chosen_classes:
            available_samples = min(user_samples, len(class_indices[cls]))
            idxs = np.random.choice(class_indices[cls], available_samples, replace=False)
            users_dict_train[user] = np.concatenate((users_dict_train[user], idxs), axis=0)
    client_datasets_filtrados = []
    for user, indices in users_dict_train.items():
        train_X = np.array([train_dataset.data[int(i)].numpy() for i in indices])
        train_y = np.array([train_dataset.targets[int(i)] for i in indices])
        train_X_filtrado, train_y_filtrado, indices_filtrados = filtrar_entropia_classe(train_X, train_y, chosen_classes_per_user[user])
        # Corrige para usar os índices filtrados corretamente
        indices_filtrados_absolutos = [int(indices[int(i)]) for i in indices_filtrados]
        client_datasets_filtrados.append(Subset(train_dataset, indices_filtrados_absolutos))
    return client_datasets_filtrados

def Cifar10NonIIDunbalance_filtrado(train_dataset, num_users, n_class=2, num_samples=2500, rate_unbalance=1.2):
    num_classes = len(np.unique(train_dataset.targets))
    labels = np.array(train_dataset.targets)
    class_indices = [np.where(labels == i)[0] for i in range(num_classes)]
    users_dict = {i: np.array([]) for i in range(num_users)}
    chosen_classes_per_user = {}  # Adiciona armazenamento para as classes escolhidas por usuário
    for user in range(num_users):
        chosen_classes = np.random.choice(range(num_classes), n_class, replace=False)
        chosen_classes_per_user[user] = chosen_classes  # Armazena as classes escolhidas
        user_samples = int(num_samples * (rate_unbalance ** user))  # Aplica o desbalanceamento
        for cls in chosen_classes:
            available_samples = min(user_samples, len(class_indices[cls]))
            idxs = np.random.choice(class_indices[cls], available_samples, replace=False)
            users_dict[user] = np.concatenate((users_dict[user], idxs), axis=0)
    client_datasets_filtrados = []
    for user, indices in users_dict.items():
        train_X = np.array([train_dataset.data[int(i)] for i in indices])
        train_y = np.array([train_dataset.targets[int(i)] for i in indices])
        # Aplica a filtragem baseada em entropia passando as classes escolhidas
        train_X_filtrado, train_y_filtrado, indices_filtrados = filtrar_entropia_classe(train_X, train_y, chosen_classes_per_user[user])
        # Corrige para usar os índices filtrados corretamente
        indices_filtrados_absolutos = [int(indices[int(i)]) for i in indices_filtrados]
        client_datasets_filtrados.append(Subset(train_dataset, indices_filtrados_absolutos))
    return client_datasets_filtrados

def Cifar100NonIIDunbalance_filtrado(train_dataset, num_users, n_class=20, num_samples=250, rate_unbalance=1.2):
    num_classes = 100  # Total de classes para CIFAR-100
    labels = np.array(train_dataset.targets)
    class_indices = [np.where(labels == i)[0] for i in range(num_classes)]
    users_dict = {i: np.array([]) for i in range(num_users)}
    chosen_classes_per_user = {}
    for user in range(num_users):
        chosen_classes = np.random.choice(range(num_classes), n_class, replace=False)
        chosen_classes_per_user[user] = chosen_classes
        user_samples = int(num_samples * (rate_unbalance ** user))  # Ajuste para desbalanceamento
        for cls in chosen_classes:
            available_samples = min(user_samples, len(class_indices[cls]))
            idxs = np.random.choice(class_indices[cls], available_samples, replace=False)
            users_dict[user] = np.concatenate((users_dict[user], idxs), axis=0)
    client_datasets_filtrados = []
    for user, indices in users_dict.items():
        train_X = np.array([train_dataset.data[int(i)] for i in indices])
        train_y = np.array([train_dataset.targets[int(i)] for i in indices])
        # Aplica a filtragem baseada em entropia passando as classes escolhidas
        train_X_filtrado, train_y_filtrado, indices_filtrados = filtrar_entropia_classe(train_X, train_y, chosen_classes_per_user[user])
        # Corrige para usar os índices filtrados corretamente
        indices_filtrados_absolutos = [int(indices[int(i)]) for i in indices_filtrados]
        client_datasets_filtrados.append(Subset(train_dataset, indices_filtrados_absolutos))
    return client_datasets_filtrados


##### [RANDOM FILTER] IDD&NON-IDD FUNCTIONS
*   Creates a federated environment with balanced and unbalanced data distributions where entropy is applied before training.


In [ ]:
from torch.utils.data import Subset
import numpy as np
from sklearn.model_selection import train_test_split

def dividir_dados(train_X, train_y, test_size=0.5, random_state=42):
    # Divisão dos dados em 50% com estratificação
    train_X, _, train_y, _ = train_test_split(train_X, train_y, test_size=test_size, random_state=random_state, stratify=train_y)
    return train_X, train_y

# Parâmetros:
# - num_users: Número de usuários/clientes.
# - n_class: Número de classes por usuário.
# - num_samples: Número base de amostras por classe para cada usuário.
# - rate_unbalance: Fator de desequilíbrio na distribuição das amostras.
def MNISTNonIID_filtrorandom(train_dataset, num_users, n_class=2, num_samples=3000, test_size=0.5, rate_unbalance=0):
    num_classes = 10  # Total de classes para MNIST
    labels = np.array(train_dataset.targets)
    class_indices = [np.where(labels == i)[0] for i in range(num_classes)]
    users_dict_train = {i: np.array([]) for i in range(num_users)}
    for user in range(num_users):
        chosen_classes = np.random.choice(range(num_classes), n_class, replace=False)
        for cls in chosen_classes:
            idxs = np.random.choice(class_indices[cls], num_samples, replace=False)
            users_dict_train[user] = np.concatenate((users_dict_train[user], idxs), axis=0)
    client_datasets = []
    for user, indices in users_dict_train.items():
        train_X = np.array([train_dataset.data[int(i)] for i in indices])
        train_y = np.array([train_dataset.targets[int(i)] for i in indices])
        train_X, train_y = dividir_dados(train_X, train_y, test_size=test_size)
        # Utiliza os índices após a divisão
        indices_absolutos = [int(indices[int(i)]) for i in range(len(train_X))]
        client_datasets.append(Subset(train_dataset, indices_absolutos))
    return client_datasets

def FashionNonIID_filtrorandom(train_dataset, num_users, n_class=2, num_samples=3000, test_size=0.5, rate_unbalance=0):
    num_classes = 10  # Total de classes para Fashion Mnist
    labels = np.array(train_dataset.targets)
    class_indices = [np.where(labels == i)[0] for i in range(num_classes)]
    users_dict_train = {i: np.array([]) for i in range(num_users)}
    for user in range(num_users):
        chosen_classes = np.random.choice(range(num_classes), n_class, replace=False)
        for cls in chosen_classes:
            idxs = np.random.choice(class_indices[cls], num_samples, replace=False)
            users_dict_train[user] = np.concatenate((users_dict_train[user], idxs), axis=0)
    client_datasets = []
    for user, indices in users_dict_train.items():
        train_X = np.array([train_dataset.data[int(i)] for i in indices])
        train_y = np.array([train_dataset.targets[int(i)] for i in indices])
        train_X, train_y = dividir_dados(train_X, train_y, test_size=test_size)
        # Utiliza os índices após a divisão
        indices_absolutos = [int(indices[int(i)]) for i in range(len(train_X))]
        client_datasets.append(Subset(train_dataset, indices_absolutos))
    return client_datasets

def Cifar10NonIID_filtrorandom(train_dataset, num_users, n_class=2, num_samples=2500, test_size=0.5, rate_unbalance=0):
    num_classes = 10  # Total de classes para Cifar10
    labels = np.array(train_dataset.targets)
    class_indices = [np.where(labels == i)[0] for i in range(num_classes)]
    users_dict_train = {i: np.array([]) for i in range(num_users)}
    for user in range(num_users):
        chosen_classes = np.random.choice(range(num_classes), n_class, replace=False)
        for cls in chosen_classes:
            idxs = np.random.choice(class_indices[cls], num_samples, replace=False)
            users_dict_train[user] = np.concatenate((users_dict_train[user], idxs), axis=0)
    client_datasets = []
    for user, indices in users_dict_train.items():
        train_X = np.array([train_dataset.data[int(i)] for i in indices])
        train_y = np.array([train_dataset.targets[int(i)] for i in indices])
        train_X, train_y = dividir_dados(train_X, train_y, test_size=test_size)
        # Utiliza os índices após a divisão
        indices_absolutos = [int(indices[int(i)]) for i in range(len(train_X))]
        client_datasets.append(Subset(train_dataset, indices_absolutos))
    return client_datasets

def Cifar100NonIID_filtrorandom(train_dataset, num_users, n_class=20, num_samples=250, test_size=0.5, rate_unbalance=0):
    num_classes = 100  # Total de classes para Cifar10
    labels = np.array(train_dataset.targets)
    class_indices = [np.where(labels == i)[0] for i in range(num_classes)]
    users_dict_train = {i: np.array([]) for i in range(num_users)}
    for user in range(num_users):
        chosen_classes = np.random.choice(range(num_classes), n_class, replace=False)
        for cls in chosen_classes:
            idxs = np.random.choice(class_indices[cls], num_samples, replace=False)
            users_dict_train[user] = np.concatenate((users_dict_train[user], idxs), axis=0)
    client_datasets = []
    for user, indices in users_dict_train.items():
        train_X = np.array([train_dataset.data[int(i)] for i in indices])
        train_y = np.array([train_dataset.targets[int(i)] for i in indices])
        train_X, train_y = dividir_dados(train_X, train_y, test_size=test_size)
        # Utiliza os índices após a divisão
        indices_absolutos = [int(indices[int(i)]) for i in range(len(train_X))]
        client_datasets.append(Subset(train_dataset, indices_absolutos))
    return client_datasets



# Parâmetros:
# - num_users: Número de usuários/clientes.
# - n_class: Número de classes por usuário.
# - num_samples: Número base de amostras por classe para cada usuário.
# - rate_unbalance: Fator de desequilíbrio na distribuição das amostras.
# - Alto fator de desbalanceamento de quanitade de classes e dados (DADOS NON-IDD).
def MNISTNonIIDunbalance_filtrorandom(train_dataset, num_users, n_class=2, num_samples=3000, rate_unbalance=1.2):
    num_classes = 10  # Total de classes para MNIST
    labels = np.array(train_dataset.targets)
    class_indices = [np.where(labels == i)[0] for i in range(num_classes)]
    users_dict_train = {i: np.array([]) for i in range(num_users)}
    for user in range(num_users):
        chosen_classes = np.random.choice(range(num_classes), n_class, replace=False)
        user_samples = int(num_samples * (rate_unbalance ** user))
        for cls in chosen_classes:
            available_samples = min(user_samples, len(class_indices[cls]))
            idxs = np.random.choice(class_indices[cls], available_samples, replace=False)
            users_dict_train[user] = np.concatenate((users_dict_train[user], idxs), axis=0)
    client_datasets = []
    for user, indices in users_dict_train.items():
        train_X = np.array([train_dataset.data[int(i)] for i in indices])
        train_y = np.array([train_dataset.targets[int(i)] for i in indices])
        train_X, train_y = dividir_dados(train_X, train_y, test_size=0.5)
        # Utiliza os índices após a divisão
        indices_absolutos = [int(indices[int(i)]) for i in range(len(train_X))]
        client_datasets.append(Subset(train_dataset, indices_absolutos))
    return client_datasets

def FashionNonIIDunbalance_filtrorandom(train_dataset, num_users, n_class=2, num_samples=3000, rate_unbalance=1.2):
    num_classes = 10  # Total de classes para Fashion MNIST
    labels = np.array(train_dataset.targets)
    class_indices = [np.where(labels == i)[0] for i in range(num_classes)]
    users_dict_train = {i: np.array([]) for i in range(num_users)}
    for user in range(num_users):
        chosen_classes = np.random.choice(range(num_classes), n_class, replace=False)
        user_samples = int(num_samples * (rate_unbalance ** user))
        for cls in chosen_classes:
            available_samples = min(user_samples, len(class_indices[cls]))
            idxs = np.random.choice(class_indices[cls], available_samples, replace=False)
            users_dict_train[user] = np.concatenate((users_dict_train[user], idxs), axis=0)
    client_datasets = []
    for user, indices in users_dict_train.items():
        train_X = np.array([train_dataset.data[int(i)] for i in indices])
        train_y = np.array([train_dataset.targets[int(i)] for i in indices])
        train_X, train_y = dividir_dados(train_X, train_y, test_size=0.5)
        # Utiliza os índices após a divisão
        indices_absolutos = [int(indices[int(i)]) for i in range(len(train_X))]
        client_datasets.append(Subset(train_dataset, indices_absolutos))
    return client_datasets

def Cifar10NonIIDunbalance_filtrorandom(train_dataset, num_users, n_class=2, num_samples=2500, rate_unbalance=1.2):
    num_classes = 10  # Total de classes para Cifar10
    labels = np.array(train_dataset.targets)
    class_indices = [np.where(labels == i)[0] for i in range(num_classes)]
    users_dict_train = {i: np.array([]) for i in range(num_users)}
    for user in range(num_users):
        chosen_classes = np.random.choice(range(num_classes), n_class, replace=False)
        user_samples = int(num_samples * (rate_unbalance ** user))
        for cls in chosen_classes:
            available_samples = min(user_samples, len(class_indices[cls]))
            idxs = np.random.choice(class_indices[cls], available_samples, replace=False)
            users_dict_train[user] = np.concatenate((users_dict_train[user], idxs), axis=0)
    client_datasets = []
    for user, indices in users_dict_train.items():
        train_X = np.array([train_dataset.data[int(i)] for i in indices])
        train_y = np.array([train_dataset.targets[int(i)] for i in indices])
        train_X, train_y = dividir_dados(train_X, train_y, test_size=0.5)
        # Utiliza os índices após a divisão
        indices_absolutos = [int(indices[int(i)]) for i in range(len(train_X))]
        client_datasets.append(Subset(train_dataset, indices_absolutos))
    return client_datasets

def Cifar100NonIIDunbalance_filtrorandom(train_dataset, num_users, n_class=20, num_samples=250, rate_unbalance=1.2):
    num_classes = 100  # Total de classes para Cifar100
    labels = np.array(train_dataset.targets)
    class_indices = [np.where(labels == i)[0] for i in range(num_classes)]
    users_dict_train = {i: np.array([]) for i in range(num_users)}
    for user in range(num_users):
        chosen_classes = np.random.choice(range(num_classes), n_class, replace=False)
        user_samples = int(num_samples * (rate_unbalance ** user))
        for cls in chosen_classes:
            available_samples = min(user_samples, len(class_indices[cls]))
            idxs = np.random.choice(class_indices[cls], available_samples, replace=False)
            users_dict_train[user] = np.concatenate((users_dict_train[user], idxs), axis=0)
    client_datasets = []
    for user, indices in users_dict_train.items():
        train_X = np.array([train_dataset.data[int(i)] for i in indices])
        train_y = np.array([train_dataset.targets[int(i)] for i in indices])
        train_X, train_y = dividir_dados(train_X, train_y, test_size=0.5)
        # Utiliza os índices após a divisão
        indices_absolutos = [int(indices[int(i)]) for i in range(len(train_X))]
        client_datasets.append(Subset(train_dataset, indices_absolutos))
    return client_datasets

#### FED-LEARNING FUNCTIONS

Collection of functions aimed at federated learning, covering global model, performance evaluation, training, and validation processes.

*   **Data Preparation (prepare_data):** This function automates the preparation of training and test datasets for MNIST, FashionMNIST, CIFAR-10, and CIFAR-100 with appropriate transformations. For CIFAR datasets, it includes data augmentation to improve model generalization.

*   **Model Selection (select_model):** Acts as a model factory, selecting and returning a specific convolutional neural network architecture based on the provided dataset name.

*   **Client Training (train_client):** Performs training of a model on the client dataset. Computes loss and accuracy, adjusting the model with local data.

*   **Model Aggregation (aggregate_models):** Implements Federated Learning aggregation logic, where model parameters from multiple clients are averaged to update the global model.

*   **Model Testing (test_model):** Evaluates the global model on the test dataset, computing loss and accuracy, and collecting predictions and true labels for further analysis.

*   **Metrics Plotting (plot_metrics, plot_confusion_matrix, plot_client_metrics, plot_all_client_metrics):** Functions dedicated to visualizing performance metrics such as loss and accuracy over epochs, as well as providing confusion matrix visualizations for detailed analysis of model performance.

In [ ]:
import torch
from torch import nn, optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Subset
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
import seaborn as sns
import time
import torch.nn.functional as F
import sys
import json
from torchvision import models

# Neural Networks
# CNN/MNIST
class CNN_Mnist(nn.Module):
    def __init__(self):
        super(CNN_Mnist, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        return F.log_softmax(self.fc2(x), dim=1)

# CNN/FashionMNIST
class CNN_FashionMnist(nn.Module):
    def __init__(self):
        super(CNN_FashionMnist, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        return F.log_softmax(self.fc2(x), dim=1)

#CNN/CIFAR-10
class CNNCifar10(nn.Module):
    def __init__(self):
        super(CNNCifar10, self).__init__()
        self.resnet = models.resnet50(pretrained=True)
        self.resnet.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.resnet.bn1 = nn.BatchNorm2d(64)
        self.resnet.maxpool = nn.Identity()
        num_ftrs = self.resnet.fc.in_features
        self.resnet.fc = nn.Sequential(
            nn.Linear(num_ftrs, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(0.5),
            nn.Linear(512, 10),  # CIFAR-10 tem 10 classes
        )
    def forward(self, x):
        return self.resnet(x)

#CNN/Cifar100
class CNNCifar100(nn.Module):
    def __init__(self):
        super(CNNCifar100, self).__init__()
        self.resnet = models.resnet50(pretrained=True)
        self.resnet.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.resnet.bn1 = nn.BatchNorm2d(64)
        self.resnet.maxpool = nn.Identity()
        num_ftrs = self.resnet.fc.in_features
        self.resnet.fc = nn.Sequential(
            nn.Linear(num_ftrs, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(0.5),
            nn.Linear(512, 100),
        )
    def forward(self, x):
        return self.resnet(x)

def save_experiment(save_name, precision, recall, f1_score, accuracy, loss, client_samples, total_samples, execution_time_per_client, total_execution_time):
    experiment_data = {
        "Global Model Metrics": {
            "Precision": precision,
            "Recall": recall,
            "F1-Score": f1_score,
            "Accuracy": accuracy,
            "Loss": loss
        },
        "Execution Information": {
            "Client Samples": client_samples,
            "Total Sample Count": total_samples,
            "Execution Time per Client (seconds)": execution_time_per_client,
            "Total Execution Time (seconds)": total_execution_time
        }
    }

    # Salvando os dados em um arquivo JSON
    with open(f"{save_name}.json", 'w') as file:
        json.dump(experiment_data, file, indent=4)
    print(f"\nWarning, the main results were saved in a file .json and can be accessed.")
    print(f"Experiment '{save_name}' saved successfully.")

def prepare_data(dataset_name, num_clients, test_batch_size, disproportion=False, distribution='idd', filter='no'):
    if distribution == 'idd' and filter != 'no':
       print("The following functions will be stopped to prevent any errors. Please define the variables correctly.")
       print("Following idd filter are allowed: 'no'.")
       sys.exit()

    if distribution=='idd':
      # Define transformações comuns, prepração de dados e etc
      normalize = transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
      if dataset_name == "CIFAR10" or dataset_name == "CIFAR100":
          # Data augmentation para CIFAR-10 e CIFAR-100
          transform_train = transforms.Compose([
              transforms.RandomHorizontalFlip(),
              transforms.RandomRotation(15),
              transforms.RandomCrop(32, padding=4),
              transforms.RandomAffine(0, translate=(0.1, 0.1)),
              transforms.ToTensor(),
              normalize,
          ])
      elif dataset_name == "MNIST" or dataset_name == "FashionMNIST":
          # Transformações padrão para MNIST e FashionMNIST com normalização específica
          normalize_mnist = transforms.Normalize((0.5,), (0.5,))
          transform_train = transforms.Compose([
              transforms.ToTensor(),
              normalize_mnist,  # Normalização específica para MNIST e FashionMNIST
          ])
      else:
          # Transformações padrão para outros datasets
          transform_train = transforms.Compose([
              transforms.ToTensor(),
              normalize,  # Normalização geral
          ])
      # Transformação para o conjunto de teste (sem data augmentation)
      transform_test = transforms.Compose([
          transforms.ToTensor(),
          normalize if dataset_name not in ["MNIST", "FashionMNIST"] else normalize_mnist,
      ])
      # Seleciona a classe do dataset com base no nome
      dataset_class = {
          "MNIST": datasets.MNIST,
          "FashionMNIST": datasets.FashionMNIST,
          "CIFAR10": datasets.CIFAR10,
          "CIFAR100": datasets.CIFAR100,
      }.get(dataset_name, datasets.MNIST)  # MNIST é o dataset padrão
      # Carrega os datasets de treino e teste com as transformações aplicadas
      train_dataset = dataset_class(root='./data', train=True, download=True, transform=transform_train)
      test_dataset = dataset_class(root='./data', train=False, download=True, transform=transform_test)
      # Divide o dataset de treino entre os clientes
      client_datasets = [Subset(train_dataset, indices.tolist()) for indices in np.array_split(range(len(train_dataset)), num_clients)]
      return client_datasets, DataLoader(test_dataset, batch_size=test_batch_size, shuffle=True)
    elif distribution=='non-idd':
      normalize_values = {
          "MNIST": transforms.Normalize((0.1307,), (0.3081,)),
          "FashionMNIST": transforms.Normalize((0.2860,), (0.3530,)),
          "CIFAR10": transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
          "CIFAR100": transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761)),
      }
      normalize = normalize_values.get(dataset_name, transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)))
      transform_train = transforms.Compose([
          transforms.ToTensor(),
          normalize,
      ])
      transform_test = transforms.Compose([
          transforms.ToTensor(),
          normalize,
      ])
      dataset_class = {
          "MNIST": datasets.MNIST,
          "FashionMNIST": datasets.FashionMNIST,
          "CIFAR10": datasets.CIFAR10,
          "CIFAR100": datasets.CIFAR100,
      }.get(dataset_name, datasets.MNIST)

      if filter == 'no':
        if dataset_name == "MNIST":
            client_datasets = MNISTNonIIDunbalance(dataset_class(root='./data', train=True, download=True, transform=transform_train),
                                          num_clients) if disproportion else MNISTNonIID(dataset_class(root='./data', train=True, download=True, transform=transform_train),
                                          num_clients)
        elif dataset_name == "FashionMNIST":
            client_datasets = FashionMNISTNonIIDunbalance(dataset_class(root='./data', train=True, download=True, transform=transform_train),
                                                num_clients) if disproportion else FashionMNISTNonIID(dataset_class(root='./data', train=True, download=True, transform=transform_train),
                                                num_clients)
        elif dataset_name == "CIFAR10":
            client_datasets = Cifar10NonIIDunbalance(dataset_class(root='./data', train=True, download=True, transform=transform_train),
                                            num_clients) if disproportion else Cifar10NonIID(dataset_class(root='./data', train=True, download=True, transform=transform_train),
                                            num_clients)
        elif dataset_name == "CIFAR100":
            client_datasets = Cifar100NonIIDunbalance(dataset_class(root='./data', train=True, download=True, transform=transform_train),
                                            num_clients) if disproportion else Cifar100NonIID(dataset_class(root='./data', train=True, download=True, transform=transform_train),
                                            num_clients)
        else:
            # Se o nome do dataset não for reconhecido, usa MNIST como padrão
            client_datasets = MNISTNonIID(dataset_class(root='./data', train=True, download=True, transform=transform_train),
                                          num_clients)
      elif filter == 'entropy':
        if dataset_name == "MNIST":
            client_datasets = MNISTNonIIDunbalance_filtrado(dataset_class(root='./data', train=True, download=True, transform=transform_train),
                                          num_clients) if disproportion else MNISTNonIID_filtrado(dataset_class(root='./data', train=True, download=True, transform=transform_train),
                                          num_clients)
        elif dataset_name == "FashionMNIST":
            client_datasets = FashionMNISTNonIIDunbalance_filtrado(dataset_class(root='./data', train=True, download=True, transform=transform_train),
                                                num_clients) if disproportion else FashionMNISTNonIID_filtrado(dataset_class(root='./data', train=True, download=True, transform=transform_train),
                                                num_clients)
        elif dataset_name == "CIFAR10":
            client_datasets = Cifar10NonIIDunbalance_filtrado(dataset_class(root='./data', train=True, download=True, transform=transform_train),
                                            num_clients) if disproportion else Cifar10NonIID_filtrado(dataset_class(root='./data', train=True, download=True, transform=transform_train),
                                            num_clients)
        elif dataset_name == "CIFAR100":
            client_datasets = Cifar100NonIIDunbalance_filtrado(dataset_class(root='./data', train=True, download=True, transform=transform_train),
                                            num_clients) if disproportion else Cifar100NonIID_filtrado(dataset_class(root='./data', train=True, download=True, transform=transform_train),
                                            num_clients)
        else:
            # Se o nome do dataset não for reconhecido, usa MNIST como padrão
            client_datasets = MNISTNonIIDunbalance_filtrado(dataset_class(root='./data', train=True, download=True, transform=transform_train),
                                          num_clients)
      elif filter == 'random':
        if dataset_name == "MNIST":
            client_datasets = MNISTNonIIDunbalance_filtrorandom(dataset_class(root='./data', train=True, download=True, transform=transform_train),
                                          num_clients) if disproportion else MNISTNonIID_filtrorandom(dataset_class(root='./data', train=True, download=True, transform=transform_train),
                                          num_clients)
        elif dataset_name == "FashionMNIST":
            client_datasets = FashionNonIIDunbalance_filtrorandom(dataset_class(root='./data', train=True, download=True, transform=transform_train),
                                                num_clients) if disproportion else FashionNonIID_filtrorandom(dataset_class(root='./data', train=True, download=True, transform=transform_train),
                                                num_clients)
        elif dataset_name == "CIFAR10":
            client_datasets = Cifar10NonIIDunbalance_filtrorandom(dataset_class(root='./data', train=True, download=True, transform=transform_train),
                                            num_clients) if disproportion else Cifar10NonIID_filtrorandom(dataset_class(root='./data', train=True, download=True, transform=transform_train),
                                            num_clients)
        elif dataset_name == "CIFAR100":
            client_datasets = Cifar100NonIIDunbalance_filtrorandom(dataset_class(root='./data', train=True, download=True, transform=transform_train),
                                            num_clients) if disproportion else Cifar100NonIID_filtrorandom(dataset_class(root='./data', train=True, download=True, transform=transform_train),
                                            num_clients)
        else:
            # Se o nome do dataset não for reconhecido, usa MNIST como padrão
            client_datasets = MNISTNonIIDunbalance_filtrorandom(dataset_class(root='./data', train=True, download=True, transform=transform_train),
                                          num_clients)
      else:
          print("Following filters are allowed: 'random', 'entropy', 'no'.")
          sys.exit()
      test_dataset = dataset_class(root='./data', train=False, download=True, transform=transform_test)
      return client_datasets, DataLoader(test_dataset, batch_size=test_batch_size, shuffle=True)
    else:
      print("Following distributions are allowed: 'non-idd', 'idd'.")
      sys.exit()

def select_model(dataset_name):
    if dataset_name == "FashionMNIST":
        return CNN_FashionMnist()
    elif dataset_name == "CIFAR10":
        return CNNCifar10()
    elif dataset_name == "CIFAR100":
        return CNNCifar100()
    else:  # Padrão para MNIST
        return CNN_Mnist()

def train_client(model, device, train_loader, optimizer, loss_function, epoch):
    # Coloca o modelo em modo de treinamento, habilitando gradientes e regularização
    print('\nStarting training round for the model...')
    model.train()
    # Inicializa contadores para a perda de treinamento, número de previsões corretas e total de amostras processadas.
    train_loss, correct, total = 0, 0, 0
    # Itera sobre o carregador de dados do cliente, processando os dados em lotes.
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = loss_function(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * data.size(0)
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
        total += target.size(0)
        if batch_idx % 10 == 0:
                total_samples = len(train_loader.dataset)
                num_digits = len(str(total_samples))
                loader_str = str(train_loader).split(' ')[-1][:-1]  # Obtém o endereço do objeto DataLoader/Client/worker
                #Print do treinamento dos clients.
                print(f'Client <{loader_str}> Train Epoch: {epoch} [{batch_idx * len(data):0{num_digits}d}/{total_samples} ({100. * batch_idx / len(train_loader):02.0f}%)]\tLoss: {loss.item():.6f}')
    accuracy = 100. * correct / total
    # Retorna a perda média de treinamento por amostra e a acurácia.
    return train_loss / len(train_loader.dataset), accuracy

def train_client_fedprox(model_local, model_global, device, train_loader, optimizer, epoch, mu=0.01):
    #Treina o modelo local de um cliente usando o algoritmo FedProx.
    print('\nStarting training round for the model...')
    model_local.train()
    train_loss, correct, total = 0, 0, 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        # Cálculo da perda incluindo o termo de proximidade
        output = model_local(data)
        classification_loss = F.cross_entropy(output, target)
        prox_term = 0.0
        for param_local, param_global in zip(model_local.parameters(), model_global.parameters()):
            prox_term += (param_local - param_global).pow(2).sum()
        prox_term *= mu
        loss = classification_loss + prox_term
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * data.size(0)
        pred = output.argmax(dim=1, keepdim=True)  # Obtém o índice da classe com a maior probabilidade
        correct += pred.eq(target.view_as(pred)).sum().item()  # Conta quantas previsões foram corretas
        total += target.size(0)
        if batch_idx % 10 == 0:
          total_samples = len(train_loader.dataset)
          num_digits = len(str(total_samples))
          loader_str = str(train_loader).split(' ')[-1][:-1]  # Obtém o endereço do objeto DataLoader/Client/worker
          #Print do treinamento dos clients.
          print(f'Client <{loader_str}> Train Epoch: {epoch} [{batch_idx * len(data):0{num_digits}d}/{total_samples} ({100. * batch_idx / len(train_loader):02.0f}%)]\tLoss: {loss.item():.6f}')
    accuracy = 100. * correct / total
    return train_loss / total, accuracy

def aggregate_models(models):
    #Modelo aggregate_models, agrega os modelos dos clientes para atualizar o modelo global.
    global_model = models[0]
    print('______________________________________________________________________________')
    print('Aggregation Step: ____________________________________________________________')
    with torch.no_grad():
        for key in global_model.state_dict().keys():
            if global_model.state_dict()[key].dtype == torch.float32:
                # Agrega os tensores, pessos e bias
                tensors_to_average = [model.state_dict()[key] for model in models]
                # Calcula os valores médio, mínimo e máximo para cada parâmetro em todos os modelos
                mean_values = torch.mean(torch.stack(tensors_to_average), dim=0).cpu().numpy().mean()
                min_values = torch.min(torch.stack(tensors_to_average), dim=0).values.cpu().numpy().min()
                max_values = torch.max(torch.stack(tensors_to_average), dim=0).values.cpu().numpy().max()
                # Imprimir estatísticas resumidas antes da agregação
                print(f"\nParameters for '{key}' aggregation:")
                print(f"Mean: {mean_values:.4f}, Min: {min_values:.4f}, Max: {max_values:.4f}")
                averaged_tensor = torch.mean(torch.stack(tensors_to_average), dim=0)
                global_model.state_dict()[key].copy_(averaged_tensor)
    return global_model

def test_model(model, device, test_loader, loss_function):
    # Coloca o modelo em modo de avaliação, desabilitando layers específicos como dropout e batch normalization.
    # Os clientes usam a função test_model para avaliar o desempenho de seus modelos locais em seus próprios dados de teste.
    model.eval()
    test_loss, correct = 0, 0
    targets_list, preds_list = [], []  # Inicializa listas para coletar targets e preds
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += loss_function(output, target).item() * data.size(0)
            pred = output.argmax(dim=1, keepdim=True)  # Predições do modelo
            correct += pred.eq(target.view_as(pred)).sum().item()

            targets_list.extend(target.view_as(pred).cpu().numpy())  # Coleta os rótulos verdadeiros
            preds_list.extend(pred.cpu().numpy())  # Coleta as previsões
    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({accuracy:.0f}%)')
    print('______________________________________________________________________________')
    return test_loss, accuracy, targets_list, preds_list  # Retorna também targets e preds

def plot_metrics(train_losses, test_losses, accuracies):
    #Plote das métricas de treino e teste ao longo das épocas.
    epochs = range(1, len(train_losses) + 1)
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_losses, 'bo-', label='Training Loss')
    plt.plot(epochs, test_losses, 'ro-', label='Test Loss')
    plt.title('Training and Test Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(epochs, accuracies, 'go-', label='Test Accuracy')
    plt.title('Test Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.legend()
    plt.tight_layout()
    plt.show()

def plot_confusion_matrix(labels, predictions):
    #Plote da matriz de confusão
    cm = confusion_matrix(labels, predictions)
    plt.figure(figsize=(10, 7))
    sns.heatmap(cm, annot=True, fmt="d", cmap='Blues', xticklabels=[str(i) for i in range(10)], yticklabels=[str(i) for i in range(10)])
    plt.xlabel('Predicted labels')
    plt.ylabel('True labels')
    plt.title('Confusion Matrix')
    plt.show()

def plot_client_metrics(client_losses, client_accuracies, client_index):
    #Plot para laço dos clients, apresenta plot de loss e acuracy de cada client.
    epochs = range(1, len(client_losses) + 1)
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, client_losses, 'bo-', label=f'Client {client_index} Training Loss')
    plt.title(f'Client {client_index} Training Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs, client_accuracies, 'go-', label=f'Client {client_index} Accuracy')
    plt.title(f'Client {client_index} Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.legend()
    plt.tight_layout()
    plt.show()

def plot_all_client_metrics(client_losses, client_accuracies):
    # Determina o número de épocas a partir do comprimento dos dados do primeiro cliente
    epochs = range(1, len(client_losses[0]) + 1)
    plt.figure(figsize=(14, 7))
    # Plot de perdas de treinamento de todos os clientes
    plt.subplot(1, 2, 1)
    for i, losses in enumerate(client_losses):
        plt.plot(epochs, losses, label=f'Client {i+1} Loss')
    plt.title('Training Loss by Client')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    # Plot de acurácias de todos os clientes
    plt.subplot(1, 2, 2)
    for i, accuracies in enumerate(client_accuracies):
        plt.plot(epochs, accuracies, label=f'Client {i+1} Accuracy')
    plt.title('Accuracy by Client')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.legend()
    plt.tight_layout()
    plt.show()

def main(dataset_name='MNIST', num_epochs=10, num_clients=5, batch_size=64, learning_rate=0.01, momentum=0.5, test_batch_size=1000, disproportion=False, distribution = 'idd', fedModel='FedAvg', mu = 0.01, filter='no', save_experiment_results = 'experiment_results'):
    # Orquestrador do ciclo completo de treinamento e avaliação em um cenário de aprendizado federado.
    # A função coordena a preparação dos dados, o treinamento dos modelos locais nos clientes e a
    # agregação dos modelos para formar um modelo global e a avaliação do modelo global.

    # Declaração de variaveis para armazenamento de informações dados e instanciamento das redes neurais, clients etc.
    start_time = time.time()  # Inicia a medição do tempo
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    client_datasets, test_loader = prepare_data(dataset_name, num_clients, test_batch_size, disproportion=disproportion, distribution=distribution, filter=filter)
    global_model = select_model(dataset_name).to(device)
    loss_function = nn.CrossEntropyLoss()
    train_losses, test_losses, accuracies = [], [], []
    all_labels, all_predictions = [], []  # Inicializa listas para coletar todos os rótulos e previsões
    client_sample_counts = [len(client_dataset) for client_dataset in client_datasets]  # Contagem de amostras por cliente
    total_samples = sum(client_sample_counts)  # Contagem total de amostras
    client_times = []
    client_losses = [[] for _ in range(num_clients)]
    client_accuracies = [[] for _ in range(num_clients)]

    # Processo de treinamento distribuído em um cenário de aprendizado federado, iterando por um número definido de épocas.
    # A cada época, todos os clientes treinam localmente uma cópia do modelo global com seus próprios dados.
    for epoch in range(1, num_epochs + 1):
        models, epoch_train_loss = [], 0
        for client_index, client_dataset in enumerate(client_datasets):
            client_start_time = time.time()  # Tempo inicial do cliente
            train_loader = DataLoader(client_dataset, batch_size=batch_size, shuffle=True)
            model_copy = type(global_model)().to(device)
            model_copy.load_state_dict(global_model.state_dict())
            optimizer = optim.SGD(model_copy.parameters(), lr=learning_rate, momentum=momentum)
            if fedModel == 'FedProx':
                mu = 0.01 if mu is None else mu
                train_loss, accuracy = train_client_fedprox(model_copy, global_model, device, train_loader, optimizer, epoch, mu)
            elif fedModel == 'FedAvg':
                train_loss, accuracy = train_client(model_copy, device, train_loader, optimizer, loss_function, epoch)
            else:
              print("Fedmodel allowed: 'FedProx', 'FedAvg'.")
            epoch_train_loss += train_loss
            models.append(model_copy)
            client_losses[client_index].append(train_loss)
            client_accuracies[client_index].append(accuracy)
            client_end_time = time.time()  # Tempo final do cliente
            client_times.append(client_end_time - client_start_time)  # Calcula o tempo de execução do cliente
        global_model = aggregate_models(models)
        train_losses.append(epoch_train_loss / len(client_datasets))

        # Chama a função test_model modificada para também retornar rótulos e previsões
        test_loss, accuracy, labels, predictions = test_model(global_model, device, test_loader, loss_function)
        test_losses.append(test_loss)
        accuracies.append(accuracy)
        # Concatena os rótulos e previsões de todas as épocas para plotar a matriz de confusão ao final
        all_labels.extend(labels)
        all_predictions.extend(predictions)

    # Plota as métricas de treinamento e teste
    print("Global Model Metrics _________________________________________________________:")
    # Converte listas de rótulos e predições para arrays numpy para cálculo
    labels_np = np.array(all_labels)
    predictions_np = np.array(all_predictions)
    # Calcula precisão, recall, F1-score e suporte para cada classe
    precision, recall, f1_score, _ = precision_recall_fscore_support(labels_np, predictions_np, average='weighted')
    # Acurácia e Loss já foram calculados e armazenados nas listas `accuracies` e `test_losses`
    accuracy_final = accuracies[-1]  # Último valor da lista de acurácias
    loss_final = test_losses[-1]  # Último valor da lista de perdas
    # Exibe os resultados
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-Score: {f1_score:.4f}")
    print(f"Accuracy: {accuracy_final:.4f}%")
    print(f"Loss: {loss_final:.4f} \n")
    plot_metrics(train_losses, test_losses, accuracies)
    # Plota a matriz de confusão usando os rótulos e previsões coletados
    print("\nGlobal Model Performance on Test Set Across All Epochs___________________________:")
    plot_confusion_matrix(all_labels, all_predictions)
    end_time = time.time()  # Tempo final total
    total_time = end_time - start_time  # Calcula o tempo total de execução

    #Printa das informações gerais dos clientes
    print("\nExecution information ___________________________________________________________:")
    for i, count in enumerate(client_sample_counts):
        print(f"Client {i+1}: {count} samples")
    print(f"Total sample count: {total_samples}")
    print(f"Execution time per client (seconds): {client_times}")
    print(f"Total execution time (seconds): {total_time}")

    print("\nClient Details ___________________________________________________________:\n")
    plot_all_client_metrics(client_losses, client_accuracies)
    print("\nIndividual Client Details ___________________________________:\n")
    for i in range(num_clients):
        plot_client_metrics(client_losses[i], client_accuracies[i], i+1)

    client_samples_info = {f"Client {i+1}": count for i, count in enumerate(client_sample_counts)}
    execution_time_per_client_info = {f"Client {i+1}": time for i, time in enumerate(client_times)}
    save_experiment(
        save_name=save_experiment_results,  # Nome do arquivo de salvamento
        precision=precision,  # Valor da precisão
        recall=recall,  # Valor do recall
        f1_score=f1_score,  # Valor do F1-Score
        accuracy=accuracy_final,  # Valor da acurácia
        loss=loss_final,  # Valor da perda
        client_samples=client_samples_info,  # Lista com a contagem de amostras por cliente
        total_samples=total_samples,  # Contagem total de amostras
        execution_time_per_client=execution_time_per_client_info,  # Lista com o tempo de execução por cliente
        total_execution_time=total_time  # Tempo total de execução
    )

### TRAINING

This class inherits from nn.Module, which forms the foundation for all neural network modules in PyTorch, specifically adapted to handle images. Thus, the main function performs model training within a federated learning context, optimizing its performance for visual data analysis and prediction.

**Class (nn.Module):**
*   Contains the architecture of the CNN used in training, which is sent to the dataloaders/workers/clients.

**Parameters of main:**
*   **dataset_name=:** Specifies that the MNIST dataset will be used for training.
*   **num_epochs:** Defines the number of training epochs.
*   **num_clients:** Configures the number of clients (or "training nodes").
*   **batch_size:** Establishes the batch size.
*   **learning_rate= and momentum=:** Defines the learning rate and momentum for the SGD optimizer.
*   **test_batch_size=:** Defines the batch size for evaluation on the test set.
*   **disproportion=:** Defines the imbalance among all participants; datasets may receive classes with different amounts of data.
*   **distribution=:** Defines whether the test will be conducted with 'iid' or 'non-iid' configuration.
*   if __name__ == "__main__": ensures that the training starts, allowing for modularization. This structure also automatically initiates the federated learning environment setup and creation of clients in DataLoaders, organizing the training sequence and managing updates to the global model.


#### Fed-Learning [MNIST]

In [ ]:
# CNN para MNIST
class CNN_Mnist(nn.Module):
    def __init__(self):
        super(CNN_Mnist, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        return F.log_softmax(self.fc2(x), dim=1)

if __name__ == "__main__":
  # If True/False, each client will have highly unbalanced datasets (unbalanced = True).
  # num_epochs defines the number of epochs for the models to train.
  # num_clients defines the number of clients.
  # batch_size defines the batch size for each client for training.
  # learning_rate defines the learning rate.
  # momentum defines the technique that controls the speed and direction of updates.
  # test_batch_size defines the batch size for testing the global model.
  # disproportion defines the high level of client data/class imbalance.
  # distribution defined as 'idd' or 'non-idd' for testing purposes.
  # fedModeledModel='', selects between FedAvg and FedProx for global model aggregation purposes.
  # Mu='', defines for FedProx the strength of penalty applied to weight deviations.
  # filter='', defines as 'yes' or 'no' for entropy filter application.
  # save_experiment_results = # define a name for the results file to be saved in .json format
    try:
        #---- Test do modelo FedAvg
        # Test Completo
        #main(dataset_name='MNIST', num_epochs=20, num_clients=10, batch_size=64, learning_rate=0.01, momentum=0.5, test_batch_size=1000, disproportion=None, distribution = 'idd', fedModel='FedAvg', mu = None, filter='no', save_experiment_results='NameHere')
        #main(dataset_name='MNIST', num_epochs=20, num_clients=10, batch_size=64, learning_rate=0.01, momentum=0.5, test_batch_size=1000, disproportion=True, distribution = 'non-idd', fedModel='FedAvg', mu = None, filter='no', save_experiment_results='NameHere')
        #main(dataset_name='MNIST', num_epochs=20, num_clients=10, batch_size=64, learning_rate=0.01, momentum=0.5, test_batch_size=1000, disproportion=False, distribution = 'non-idd', fedModel='FedAvg', mu = None, filter='no', save_experiment_results='NameHere')
        #---- Test do modelo FedProx
        #main(dataset_name='MNIST', num_epochs=20, num_clients=10, batch_size=64, learning_rate=0.01, momentum=0.5, test_batch_size=1000, disproportion=None, distribution = 'idd', fedModel='FedProx', mu = 0.01, filter='no', save_experiment_results='NameHere')
        #main(dataset_name='MNIST', num_epochs=20, num_clients=10, batch_size=64, learning_rate=0.01, momentum=0.5, test_batch_size=1000, disproportion=True, distribution = 'non-idd', fedModel='FedProx', mu = 0.01, filter='no', save_experiment_results='NameHere')
        #main(dataset_name='MNIST', num_epochs=20, num_clients=10, batch_size=64, learning_rate=0.01, momentum=0.5, test_batch_size=1000, disproportion=False, distribution = 'non-idd', fedModel='FedProx', mu = 0.01, filter='no', save_experiment_results='NameHere')
        #-----
        # Test Rapido
        #main(dataset_name='MNIST', num_epochs=1, num_clients=3, batch_size=64, learning_rate=0.01, momentum=0.5, test_batch_size=1000, disproportion=None, distribution = 'idd', fedModel='FedAvg', mu = None, filter='no', save_experiment_results='NameHere')
        #main(dataset_name='MNIST', num_epochs=1, num_clients=3, batch_size=64, learning_rate=0.01, momentum=0.5, test_batch_size=1000, disproportion=True, distribution = 'non-idd', fedModel='FedAvg', mu = None, filter='no', save_experiment_results='NameHere')
        #main(dataset_name='MNIST', num_epochs=1, num_clients=3, batch_size=64, learning_rate=0.01, momentum=0.5, test_batch_size=1000, disproportion=False, distribution = 'non-idd', fedModel='FedAvg', mu = None, filter='no', save_experiment_results='NameHere')
        #main(dataset_name='MNIST', num_epochs=1, num_clients=3, batch_size=64, learning_rate=0.01, momentum=0.5, test_batch_size=1000, disproportion=None, distribution = 'idd', fedModel='FedProx', mu = 0.01, filter='no', save_experiment_results='NameHere')
        #main(dataset_name='MNIST', num_epochs=1, num_clients=3, batch_size=64, learning_rate=0.01, momentum=0.5, test_batch_size=1000, disproportion=True, distribution = 'non-idd', fedModel='FedProx', mu = 0.01, filter='no', save_experiment_results='NameHere')
        #main(dataset_name='MNIST', num_epochs=1, num_clients=3, batch_size=64, learning_rate=0.01, momentum=0.5, test_batch_size=1000, disproportion=False, distribution = 'non-idd', fedModel='FedProx', mu = 0.01, filter='no', save_experiment_results='NameHere')
        pass
    except KeyboardInterrupt:
        print("\nTraining interrupted.")

#### Fed-Learning [Fashion Mnist]

In [ ]:
# CNN para FashionMNIST
class CNN_FashionMnist(nn.Module):
    def __init__(self):
        super(CNN_FashionMnist, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        return F.log_softmax(self.fc2(x), dim=1)

if __name__ == "__main__":
  # If True/False, each client will have highly unbalanced datasets (unbalanced = True).
  # num_epochs defines the number of epochs for the models to train.
  # num_clients defines the number of clients.
  # batch_size defines the batch size for each client for training.
  # learning_rate defines the learning rate.
  # momentum defines the technique that controls the speed and direction of updates.
  # test_batch_size defines the batch size for testing the global model.
  # disproportion defines the high level of client data/class imbalance.
  # distribution defined as 'idd' or 'non-idd' for testing purposes.
  # fedModeledModel='', selects between FedAvg and FedProx for global model aggregation purposes.
  # Mu='', defines for FedProx the strength of penalty applied to weight deviations.
  # filter='', defines as 'yes' or 'no' for entropy filter application.
  # save_experiment_results = # define a name for the results file to be saved in .json format
    try:
        #---- Test do modelo FedAvg
        # Test Completo
        #main(dataset_name='FashionMNIST', num_epochs=20, num_clients=10, batch_size=64, learning_rate=0.01, momentum=0.5, test_batch_size=1000, disproportion=None, distribution = 'idd', fedModel='FedAvg', mu = None, filter='no', save_experiment_results='NameHere')
        #main(dataset_name='FashionMNIST', num_epochs=20, num_clients=10, batch_size=64, learning_rate=0.01, momentum=0.5, test_batch_size=1000, disproportion=True, distribution = 'non-idd', fedModel='FedAvg', mu = None, filter='no', save_experiment_results='NameHere')
        #main(dataset_name='FashionMNIST', num_epochs=20, num_clients=10, batch_size=64, learning_rate=0.01, momentum=0.5, test_batch_size=1000, disproportion=False, distribution = 'non-idd', fedModel='FedAvg', mu = None, filter='no', save_experiment_results='NameHere')
        #----- Test do modelo FedProx
        #main(dataset_name='FashionMNIST', num_epochs=20, num_clients=10, batch_size=64, learning_rate=0.01, momentum=0.5, test_batch_size=1000, disproportion=None, distribution = 'idd', fedModel='FedProx', mu = 0.01, filter='no', save_experiment_results='NameHere')
        #main(dataset_name='FashionMNIST', num_epochs=20, num_clients=10, batch_size=64, learning_rate=0.01, momentum=0.5, test_batch_size=1000, disproportion=True, distribution = 'non-idd', fedModel='FedProx', mu = 0.01, filter='no', save_experiment_results='NameHere')
        #main(dataset_name='FashionMNIST', num_epochs=20, num_clients=10, batch_size=64, learning_rate=0.01, momentum=0.5, test_batch_size=1000, disproportion=False, distribution = 'non-idd', fedModel='FedProx', mu = 0.01, filter='no', save_experiment_results='NameHere')

        #-----
        # Test Rapido
        #main(dataset_name='FashionMNIST', num_epochs=1, num_clients=3, batch_size=64, learning_rate=0.01, momentum=0.5, test_batch_size=1000, disproportion=None, distribution = 'idd', fedModel='FedAvg', mu = None, filter='no', save_experiment_results='NameHere')
        #main(dataset_name='FashionMNIST', num_epochs=1, num_clients=3, batch_size=64, learning_rate=0.01, momentum=0.5, test_batch_size=1000, disproportion=True, distribution = 'non-idd', fedModel='FedAvg', mu = None, filter='no', save_experiment_results='NameHere')
        #main(dataset_name='FashionMNIST', num_epochs=1, num_clients=3, batch_size=64, learning_rate=0.01, momentum=0.5, test_batch_size=1000, disproportion=False, distribution = 'non-idd', fedModel='FedAvg', mu = None, filter='no', save_experiment_results='NameHere')
        #main(dataset_name='FashionMNIST', num_epochs=1, num_clients=3, batch_size=64, learning_rate=0.01, momentum=0.5, test_batch_size=1000, disproportion=None, distribution = 'idd', fedModel='FedProx', mu = 0.01, filter='no', save_experiment_results='NameHere')
        #main(dataset_name='FashionMNIST', num_epochs=1, num_clients=3, batch_size=64, learning_rate=0.01, momentum=0.5, test_batch_size=1000, disproportion=True, distribution = 'non-idd', fedModel='FedProx', mu = 0.01, filter='no', save_experiment_results='NameHere')
        #main(dataset_name='FashionMNIST', num_epochs=1, num_clients=3, batch_size=64, learning_rate=0.01, momentum=0.5, test_batch_size=1000, disproportion=False, distribution = 'non-idd', fedModel='FedProx', mu = 0.01, filter='no', save_experiment_results='NameHere')
        pass
    except KeyboardInterrupt:
        print("\nTraining interrupted.")

#### Fed-Learning [CIFAR-10]

In [ ]:
#CNN/CIFAR-10
class CNNCifar10(nn.Module):
    def __init__(self):
        super(CNNCifar10, self).__init__()
        self.resnet = models.resnet50(pretrained=True)
        self.resnet.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.resnet.bn1 = nn.BatchNorm2d(64)
        self.resnet.maxpool = nn.Identity()
        num_ftrs = self.resnet.fc.in_features
        self.resnet.fc = nn.Sequential(
            nn.Linear(num_ftrs, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(0.5),
            nn.Linear(512, 10),  # CIFAR-10 tem 10 classes
        )
    def forward(self, x):
        return self.resnet(x)

if __name__ == "__main__":
    # If True/False, each client will have highly unbalanced datasets (unbalanced = True).
    # num_epochs defines the number of epochs for the models to train.
    # num_clients defines the number of clients.
    # batch_size defines the batch size for each client for training.
    # learning_rate defines the learning rate.
    # momentum defines the technique that controls the speed and direction of updates.
    # test_batch_size defines the batch size for testing the global model.
    # disproportion defines the high level of client data/class imbalance.
    # distribution defined as 'idd' or 'non-idd' for testing purposes.
    # fedModeledModel='', selects between FedAvg and FedProx for global model aggregation purposes.
    # Mu='', defines for FedProx the strength of penalty applied to weight deviations.
    # filter='', defines as 'yes' or 'no' for entropy filter application.
    # save_experiment_results = # define a name for the results file to be saved in .json format
    try:
        #main(dataset_name='CIFAR10', num_epochs=50, num_clients=10, batch_size=64, learning_rate=0.01, momentum=0.5, test_batch_size=1000, disproportion=True, distribution = 'non-idd', fedModel='FedAvg', mu = None, filter='no', save_experiment_results='CIFAR10_TRUENONIDD_FEDAVG_FILTERNO_10')
        # Test Completo
        #main(dataset_name='CIFAR10', num_epochs=50, num_clients=10, batch_size=64, learning_rate=0.01, momentum=0.5, test_batch_size=1000, disproportion=None, distribution = 'idd', fedModel='FedAvg', mu = None, filter='no', save_experiment_results='NameHere')
        #main(dataset_name='CIFAR10', num_epochs=50, num_clients=10, batch_size=64, learning_rate=0.01, momentum=0.5, test_batch_size=1000, disproportion=True, distribution = 'non-idd', fedModel='FedAvg', mu = None, filter='no', save_experiment_results='NameHere')
        #main(dataset_name='CIFAR10', num_epochs=50, num_clients=10, batch_size=64, learning_rate=0.01, momentum=0.5, test_batch_size=1000, disproportion=False, distribution = 'non-idd', fedModel='FedAvg', mu = None, filter='no', save_experiment_results='NameHere')
        #---- Test do modelo FedProx
        #main(dataset_name='CIFAR10', num_epochs=50, num_clients=10, batch_size=64, learning_rate=0.01, momentum=0.5, test_batch_size=1000, disproportion=None, distribution = 'idd', fedModel='FedProx', mu = 0.01, filter='no', save_experiment_results='NameHere')
        #main(dataset_name='CIFAR10', num_epochs=50, num_clients=10, batch_size=64, learning_rate=0.01, momentum=0.5, test_batch_size=1000, disproportion=True, distribution = 'non-idd', fedModel='FedProx', mu = 0.01, filter='no', save_experiment_results='NameHere')
        #main(dataset_name='CIFAR10', num_epochs=50, num_clients=10, batch_size=64, learning_rate=0.01, momentum=0.5, test_batch_size=1000, disproportion=False, distribution = 'non-idd', fedModel='FedProx', mu = 0.01, filter='no', save_experiment_results='NameHere')
        #-----
        # Test Rapido
        #main(dataset_name='CIFAR10', num_epochs=1, num_clients=3, batch_size=64, learning_rate=0.01, momentum=0.5, test_batch_size=1000, disproportion=None, distribution = 'idd', fedModel='FedAvg', mu = None, filter='no', save_experiment_results='NameHere')
        #main(dataset_name='CIFAR10', num_epochs=1, num_clients=3, batch_size=64, learning_rate=0.01, momentum=0.5, test_batch_size=1000, disproportion=True, distribution = 'non-idd', fedModel='FedAvg', mu = None, filter='no', save_experiment_results='NameHere')
        #main(dataset_name='CIFAR10', num_epochs=1, num_clients=3, batch_size=64, learning_rate=0.01, momentum=0.5, test_batch_size=1000, disproportion=False, distribution = 'non-idd', fedModel='FedAvg', mu = None, filter='no', save_experiment_results='NameHere')
        #main(dataset_name='CIFAR10', num_epochs=1, num_clients=3, batch_size=64, learning_rate=0.01, momentum=0.5, test_batch_size=1000, disproportion=None, distribution = 'idd', fedModel='FedProx', mu = 0.01, filter='no', save_experiment_results='NameHere')
        #main(dataset_name='CIFAR10', num_epochs=1, num_clients=3, batch_size=64, learning_rate=0.01, momentum=0.5, test_batch_size=1000, disproportion=True, distribution = 'non-idd', fedModel='FedProx', mu = 0.01, filter='no', save_experiment_results='NameHere')
        #main(dataset_name='CIFAR10', num_epochs=1, num_clients=3, batch_size=64, learning_rate=0.01, momentum=0.5, test_batch_size=1000, disproportion=False, distribution = 'non-idd', fedModel='FedProx', mu = 0.01, filter='no', save_experiment_results='NameHere')
        pass
    except KeyboardInterrupt:
        print("\nTraining interrupted.")

#### Fed-Learning [CIFAR-100]

In [ ]:
# Definição da classe CNNCifar100
class CNNCifar100(nn.Module):
    def __init__(self):
        super(CNNCifar100, self).__init__()
        self.resnet = models.resnet50(pretrained=True)
        self.resnet.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.resnet.bn1 = nn.BatchNorm2d(64)
        self.resnet.maxpool = nn.Identity()
        num_ftrs = self.resnet.fc.in_features
        self.resnet.fc = nn.Sequential(
            nn.Linear(num_ftrs, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(0.5),
            nn.Linear(512, 100),
        )
    def forward(self, x):
        return self.resnet(x)

if __name__ == "__main__":
    # If True/False, each client will have highly unbalanced datasets (unbalanced = True).
    # num_epochs defines the number of epochs for the models to train.
    # num_clients defines the number of clients.
    # batch_size defines the batch size for each client for training.
    # learning_rate defines the learning rate.
    # momentum defines the technique that controls the speed and direction of updates.
    # test_batch_size defines the batch size for testing the global model.
    # disproportion defines the high level of client data/class imbalance.
    # distribution defined as 'idd' or 'non-idd' for testing purposes.
    # fedModeledModel='', selects between FedAvg and FedProx for global model aggregation purposes.
    # Mu='', defines for FedProx the strength of penalty applied to weight deviations.
    # filter='', defines as 'yes' or 'no' for entropy filter application.
    # save_experiment_results = # define a name for the results file to be saved in .json format
    try:
      #----- Test do modelo FedAvg
      #main(dataset_name='CIFAR100', num_epochs=20, num_clients=10, batch_size=64, learning_rate=0.01, momentum=0.5, test_batch_size=1000, disproportion=None, distribution = 'idd', fedModel='FedAvg', mu = None, filter='no', save_experiment_results='NameHere')
      #main(dataset_name='CIFAR100', num_epochs=20, num_clients=10, batch_size=64, learning_rate=0.01, momentum=0.5, test_batch_size=1000, disproportion=True, distribution = 'non-idd', fedModel='FedAvg', mu = None, filter='no', save_experiment_results='NameHere')
      #main(dataset_name='CIFAR100', num_epochs=20, num_clients=5, batch_size=64, learning_rate=0.01, momentum=0.5, test_batch_size=1000, disproportion=False, distribution = 'non-idd', fedModel='FedAvg', mu = None, filter='no', save_experiment_results='NameHere')
      #----- Test do modelo FedProx
      #main(dataset_name='CIFAR100', num_epochs=20, num_clients=10, batch_size=64, learning_rate=0.01, momentum=0.5, test_batch_size=1000, disproportion=None, distribution = 'idd', fedModel='FedProx', mu = 0.01, filter='no', save_experiment_results='NameHere')
      #main(dataset_name='CIFAR100', num_epochs=20, num_clients=10, batch_size=64, learning_rate=0.01, momentum=0.5, test_batch_size=1000, disproportion=True, distribution = 'non-idd', fedModel='FedProx', mu = 0.01, filter='no', save_experiment_results='NameHere')
      #main(dataset_name='CIFAR100', num_epochs=20, num_clients=5, batch_size=64, learning_rate=0.01, momentum=0.5, test_batch_size=1000, disproportion=False, distribution = 'non-idd', fedModel='FedProx', mu = 0.01, filter='no', save_experiment_results='NameHere')

      #-----
      # Test Rapido
      #main(dataset_name='CIFAR100', num_epochs=1, num_clients=2, batch_size=64, learning_rate=0.01, momentum=0.5, test_batch_size=1000, disproportion=None, distribution = 'idd', fedModel='FedAvg', mu = None, filter='no', save_experiment_results='NameHere')
      #main(dataset_name='CIFAR100', num_epochs=1, num_clients=2, batch_size=64, learning_rate=0.01, momentum=0.5, test_batch_size=1000, disproportion=True, distribution = 'non-idd', fedModel='FedAvg', mu = None, filter='random', save_experiment_results='NameHere')
      #main(dataset_name='CIFAR100', num_epochs=1, num_clients=2, batch_size=64, learning_rate=0.01, momentum=0.5, test_batch_size=1000, disproportion=False, distribution = 'non-idd', fedModel='FedAvg', mu = None, filter='random', save_experiment_results='NameHere')
      #main(dataset_name='CIFAR100', num_epochs=1, num_clients=2, batch_size=64, learning_rate=0.01, momentum=0.5, test_batch_size=1000, disproportion=None, distribution = 'idd', fedModel='FedProx', mu = 0.01, filter='no', save_experiment_results='NameHere')
      #main(dataset_name='CIFAR100', num_epochs=1, num_clients=2, batch_size=64, learning_rate=0.01, momentum=0.5, test_batch_size=1000, disproportion=True, distribution = 'non-idd', fedModel='FedProx', mu = 0.01, filter='random', save_experiment_results='NameHere')
      #main(dataset_name='CIFAR100', num_epochs=1, num_clients=2, batch_size=64, learning_rate=0.01, momentum=0.5, test_batch_size=1000, disproportion=False, distribution = 'non-idd', fedModel='FedProx', mu = 0.01, filter='random', save_experiment_results='NameHere')
      pass
    except KeyboardInterrupt:
        print("\nTraining interrupted.")